In [3]:
import pandas as pd

import matplotlib.ticker as tick
import matplotlib.pylab as plt
import seaborn as sns
import plotly.express as px
from util.reformat_large_tick_values import reformat_large_tick_values as ticker

from sqlalchemy.engine import create_engine
import warnings
warnings.filterwarnings('ignore', message='Cannot create BigQuery Storage client*')

In [4]:
engine = create_engine('bigquery://', credentials_path= 'key/iowa-liquor-sales-365322-12ff7a5498e6.json')

In [6]:
counties = ['Polk', 'Linn', 'Scott', 'Johnson', 'Black Hawk', 'Woodbury', 'Dubuque', 'Story', 'Dallas', 'Pottawattamie']
colors = ['#edc948', '#59a14f', '#ff9da7', '#76b7b2', '#4e79a7', '#b07aa1', '#bab0ac', '#9c755f', '#e15759', '#f28e2b']
county_colors = {county: color for county, color in zip(counties, colors)}

# Data imports

Liquor Sales Data: Iowa Liquor Sales dataset, Iowa Department of Commerce, Alcoholic Beverages Division
 `bigquery-public-data.iowa_liquor_sales`  
Data represents sales to Iowa Class "E" liquor licensees. Examples include grocery stores, liquor stores etc which sell liquor for off-premises consumption. Full dataset stored in BigQuery. Data accessed via SQL queries.

Population Data: United States Census Bureau, https://www.census.gov/quickfacts/fact/table/IA/POP010220 
Accurate population numbers are limited to 2020, the year of the US census. In this analysis, population counts and percent minors for 2020 are used for all years (2018-2023). Adult population was extrapolated using the "Persons under 18 years, percent" census data column as no counts are available for 21 years and older.

Data imports from source and dataframe creation available in [Iowa Liquor Exploratory Analysis](iowa_liquor_exploratory_analysis.ipynb)

In [8]:
liters_sold_per_adult_df = pd.read_csv('liters_sold_per_adult.csv')
liters_sold_per_adult_df

,county,liters_sold_per_adult
0,Polk,80.510252
1,Black Hawk,72.315520
2,Scott,70.322979
3,Pottawattamie,68.566631
4,Linn,68.438600
5,Johnson,62.911275
6,Woodbury,60.300233
7,Dubuque,55.686160
8,Story,51.906894
9,Dallas,48.841635


In [ ]:
liters_county_month_df = pd.read_csv('